In [1]:
import ROOT
from ROOT import gROOT, TCanvas, TF1, TFile, TTree, gRandom, TH1F

from ROOT import RooRealVar, RooFormulaVar, RooVoigtian, RooChebychev, RooArgList, \
                 RooArgSet, RooAddPdf, RooDataSet, RooCategory, RooSimultaneous, \
                 RooBreitWigner, RooCBShape, RooFFTConvPdf, RooGaussian,RooExponential, \
                 RooBinning, kRed, kBlue, kDotted,TString,RooAbsData, RooPlot, TCut, RooAbsData, RooFit , RooStats               

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os, sys, time, random

from ROOT import TTree, TFile

# from root_numpy import root2array, rec2array, array2root

import pandas as pd
import numpy as np
import scipy 
import root_pandas as rp
import root_numpy as ry 

import pandas.core.common as com
from pandas.core.index import Index
from pandas.tools import plotting
from pandas.tools.plotting import scatter_matrix

from tqdm import tqdm_notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.externals import joblib
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score

sys.path.append('/home/chasenberg/repos/')
sys.path.append('/home/chasenberg/repos/dopy')
from dopy import * 
from dopy.dolearn.sklearn_utils import plot_roc_curve, plot_classifier_output, plot_correlations
from dopy.dolearn.sklearn_utils import plot_feature_importances, plot_classifier_output, classify_unseen_data
#from dopy.sklearn_utils import plot_bdt_vars
from dopy.doplot.plotting import Plotter, Plot
from dopy.doanalysis.df_utils import add_min_max, add_eta 

/opt/rh/miniconda/envs/py3root6/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
sys.path.append('/home/chasenberg/repos/root_utils/')
from ROOT import TColor
import rootnotes  # For displaying TCanvas in notebooks
from lhcb_style import set_lhcb_style # Set basic LHCb plot style
set_lhcb_style()   
from root_utils import plot_pulls

In [4]:

from ROOT import RooRealVar, RooExponential, RooExtendPdf, RooAddPdf, RooArgList, RooGaussian
from ROOT import RooFormulaVar
from ROOT import gSystem
gSystem.Load('libRooFit.so')
gSystem.Load('/home/chasenberg/repos/dopy/dolib/CustomShapes/libKll')
from ROOT import RooIpatia2

In [5]:
cut = '(B0_FitDaughtersConst_M_flat<5450&B0_FitDaughtersConst_M_flat>5220)&B0_FitPVConst_tau_flat>0.0002&B0_FitPVConst_tau_flat<0.015'
#cut = ''
df = rp.read_root("/fhgfs/users/chasenberg/data/2015_2016_merged/jpsimumuks/Bd2JpsiKS_final.root",where=cut, key="Bd2JpsiKS")
df.to_root("/fhgfs/users/chasenberg/data/2015_2016_merged/jpsimumuks/Bd2JpsiKS_final_cuts.root", key="Bd2JpsiKS")

## Choose if massfit and sWeight calculation is performed on random or best selected data

In [6]:
random_sel_massfit = True

In [7]:
# Read data and fit it
dataset_file_path = '/fhgfs/users/chasenberg/data/2015_2016_merged/jpsimumuks/'

if random_sel_massfit==True:
    file_name = 'Bd2JpsiKS_final_cuts.root'
    file_name_sweight = 'Bd2JpsiKS_final_sw.root'
else:
    file_name = 'Bd2JpsiKS_bestPV_final.root'
    file_name_sweight = 'Bd2JpsiKS_bestPV_final_sw.root'
data = ROOT.TFile(dataset_file_path + file_name)
tree_data = data.Get('Bd2JpsiKS') 

In [8]:
tree_data.GetEntries()

173332

In [9]:
#Mass variable
mass         = RooRealVar("B0_FitDaughtersConst_M_flat", "m_{J/psi K_{S}^{0}}", 5280,5220, 5450, "MeV")
ntupleVarSet =  RooArgSet(mass)#,nsig_sw,nbkg_sw) 
dataset      = RooDataSet('data','data',tree_data,ntupleVarSet)

Build Ipatia mass model
--------------------------------------------

In [10]:
postfix = ''

par_ipatia_zeta  = RooRealVar('par_ipatia_zeta'  + postfix,  'par_ipatia_zeta'  + postfix, 3.25588)#, 0.3, 5.3)
par_ipatia_fb    = RooRealVar('par_ipatia_fb'    + postfix,    'par_ipatia_fb'  + postfix, 0.)#, 0., 10.)
par_ipatia_l     = RooRealVar('par_ipatia_l'     + postfix,     'par_ipatia_l'  + postfix, -3.17)#1., 1., 1.)
par_ipatia_m     = RooRealVar('par_ipatia_m'     + postfix,     'par_ipatia_m'  + postfix, 5279, 5200., 5400.,)
par_ipatia_sigma = RooRealVar('par_ipatia_sigma' + postfix, 'par_ipatia_sigma'  + postfix, 7.94441, 2., 100.)
par_ipatia_a1    = RooRealVar('par_ipatia_a1'    + postfix,     'par_ipatia_a1' + postfix, 1.76044)#, 0.01, 4)
par_ipatia_a2    = RooRealVar('par_ipatia_a2'    + postfix,     'par_ipatia_a2' + postfix, 1.88748)#, 0.01, 4)
par_ipatia_n1    = RooRealVar('par_ipatia_n1'    + postfix,     'par_ipatia_n1' + postfix, 4.76854)#, 0.00001, 150.)
par_ipatia_n2    = RooRealVar('par_ipatia_n2'    + postfix,     'par_ipatia_n2' + postfix, 6.76837)#, 0.00001, 150.)'''

par_ipatia_zeta  = RooRealVar('par_ipatia_zeta'  + postfix,  'par_ipatia_zeta'  + postfix, 0.,0.,0.)#, 1., 5.3)
par_ipatia_fb    = RooRealVar('par_ipatia_fb'    + postfix,    'par_ipatia_fb'  + postfix, 0.,0.,0.)
par_ipatia_l     = RooRealVar('par_ipatia_l'     + postfix,     'par_ipatia_l'  + postfix, -2.718,-2.718,-2.718)
par_ipatia_m     = RooRealVar('par_ipatia_m'     + postfix,     'par_ipatia_m'  + postfix, 5280.5, 5220., 5400.,)
par_ipatia_sigma = RooRealVar('par_ipatia_sigma' + postfix, 'par_ipatia_sigma'  + postfix, 8.71582, 2., 12.)
par_ipatia_a1    = RooRealVar('par_ipatia_a1'    + postfix,     'par_ipatia_a1' + postfix,  3.,3.,3.)
par_ipatia_a2    = RooRealVar('par_ipatia_a2'    + postfix,     'par_ipatia_a2' + postfix,  2.91222, 2.91222, 2.91222)
par_ipatia_n1    = RooRealVar('par_ipatia_n1'    + postfix,     'par_ipatia_n1' + postfix, 3.,3.,3.)
par_ipatia_n2    = RooRealVar('par_ipatia_n2'    + postfix,     'par_ipatia_n2' + postfix, 3.,3.,3.)

pdf_mass_ipatia  = ROOT.RooIpatia2('pdf_mass_ipatia', 'pdf_mass_ipatia', mass, par_ipatia_l, par_ipatia_zeta, par_ipatia_fb,
                               par_ipatia_sigma, par_ipatia_m, par_ipatia_a1, par_ipatia_n1,
                               par_ipatia_a2, par_ipatia_n2)
    
ROOT.SetOwnership( pdf_mass_ipatia, False )
ROOT.SetOwnership( par_ipatia_zeta, False )
ROOT.SetOwnership( par_ipatia_fb, False )
ROOT.SetOwnership( par_ipatia_l, False )
ROOT.SetOwnership( par_ipatia_m, False )
ROOT.SetOwnership( par_ipatia_sigma, False )
ROOT.SetOwnership( par_ipatia_a1, False )
ROOT.SetOwnership( par_ipatia_n1, False )
ROOT.SetOwnership( par_ipatia_a2, False )
ROOT.SetOwnership( par_ipatia_n2, False )

sig_yield = RooRealVar('sig_yield' + postfix, 'sig_yield' + postfix, 160000, 0, 250000)
ROOT.SetOwnership( sig_yield, False )

sig_pdf_ext = RooExtendPdf('sig_pdf_ext' + postfix, 'sig_pdf_ext' + postfix, pdf_mass_ipatia, sig_yield)
ROOT.SetOwnership( sig_pdf_ext, False )
 
exp_a = RooRealVar('exp_a' + postfix, 'exp_a' + postfix, -0.001, -0.01, -0.00001)
exp   = RooExponential('exp' + postfix, 'exp' + postfix, mass, exp_a)
bkg_pdf = exp
bkg_yield = RooRealVar('bkg_yield' + postfix, 'bkg_yield' + postfix, 1849830, 0, 2000000)
ROOT.SetOwnership( exp_a, False )
ROOT.SetOwnership( bkg_pdf, False )
ROOT.SetOwnership( bkg_yield, False)

bkg_pdf_ext = RooExtendPdf('bkg_pdf_ext' + postfix, 'bkg_pdf_ext' + postfix, bkg_pdf, bkg_yield)
ROOT.SetOwnership( bkg_pdf_ext, False)

par_ipatia_m_bs_shift = RooRealVar("par_ipatia_m_bs_shift" + postfix, "par_ipatia_m_bs_shift" + postfix, 87.0, 50.0, 100.0)
par_ipatia_m_bs = RooFormulaVar("par_ipatia_m_bs" + postfix, "par_ipatia_m_bs" + postfix, "@0+@1", RooArgList(par_ipatia_m, par_ipatia_m_bs_shift))
#par_ipatia_m_bs = RooRealVar("par_ipatia_m_bs"    + postfix,    "par_ipatia_m_bs" + postfix,    5366, 5300., 5400.,)
                    
pdf_mass_ipatia_bs  = ROOT.RooIpatia2('pdf_mass_ipatia_bs', 'pdf_mass_ipatia_bs', mass, par_ipatia_l, par_ipatia_zeta, par_ipatia_fb,
                                   par_ipatia_sigma, par_ipatia_m_bs, par_ipatia_a1, par_ipatia_n1, par_ipatia_a2, par_ipatia_n2)
    
bs_yield = RooRealVar('bs_yield' + postfix, 'bs_yield' + postfix, 4794, 0, 40000)
ROOT.SetOwnership( par_ipatia_m_bs_shift, False )
ROOT.SetOwnership( par_ipatia_m_bs, False)
ROOT.SetOwnership( bs_yield, False)
ROOT.SetOwnership( pdf_mass_ipatia_bs, False)

bs_pdf_ext = RooExtendPdf('bs_pdf_ext' + postfix, 'bs_pdf_ext' + postfix, pdf_mass_ipatia_bs, bs_yield)
ROOT.SetOwnership( bs_pdf_ext, False)

data_model = RooAddPdf('model' + postfix, 'model' + postfix, RooArgList(sig_pdf_ext, bkg_pdf_ext, bs_pdf_ext))
ROOT.SetOwnership( data_model, False )

In [11]:
#Fit model to dataset
data_model.fitTo(dataset, RooFit.NumCPU(6),
                                      RooFit.Minimizer('Minuit','minimize'),
                                      RooFit.Hesse(True),
                                      RooFit.Optimize(0),
                                      RooFit.PrintEvalErrors(0),
                                      RooFit.Save(True),
                                      RooFit.Extended(),
                                      RooFit.Verbose(True)) 

<ROOT.RooFitResult object ("fitresult_model_data") at 0x971d640>

In [12]:
'''# Prepare frame
%matplotlib inline  
import ROOT




frame = mass.frame(ROOT.RooFit.Bins(130))

dataset.plotOn(frame, ROOT.RooFit.Name("data1"))

data_model.plotOn( frame , ROOT.RooFit.Components("sig_pdf_ext"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kRed))
data_model.plotOn( frame , ROOT.RooFit.Components("bkg_pdf_ext"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kBlue))
data_model.plotOn( frame , ROOT.RooFit.Components("bs_pdf_ext"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kBlue))
data_model.plotOn(frame, ROOT.RooFit.Name("fit"))
### Add additional informations to the plot
text_size = 0.035
# Create TLegend
#legend = ROOT.TLegend(0.7, 0.75, 0.9, 0.9, '')
legend = ROOT.TLegend(0.7, 0.7,0.85,0.85)
legend.AddEntry(0,"LHCb unofficial", "")
legend.AddEntry(frame.findObject('data1'), "Data", "p");
legend.AddEntry(frame.findObject('sig_pdf_ext'), 'Massmodel', 'kBluw')

#legend.AddEntry(frame.findObject('bs_pdf_ext'), 'bs_pdf_ext', 'l')
#legend.AddEntry(frame.findObject('bkg_pdf_ext'), 'bkg_pdf_ext', 'l')
legend.SetTextSize(text_size)
frame.GetYaxis().SetTitle(frame.GetYaxis().GetTitle().replace("Events", "Candidates"))
# Plot pulls
can, _ = plot_pulls('Hallo', frame, legend=legend, logy=True)#, latex=latex)
if random_sel_massfit==True:
    can.SaveAs("/home/chasenberg/plots/massfit/B0_Mass_selected_randomPV.pdf")
else:
    can.SaveAs("/home/chasenberg/plots/massfit/B0_Mass_selected_bestPV.pdf")
can  # To display plot in notebooks'''

'# Prepare frame\n%matplotlib inline  \nimport ROOT\n\n\n\n\nframe = mass.frame(ROOT.RooFit.Bins(130))\n\ndataset.plotOn(frame, ROOT.RooFit.Name("data1"))\n\ndata_model.plotOn( frame , ROOT.RooFit.Components("sig_pdf_ext"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kRed))\ndata_model.plotOn( frame , ROOT.RooFit.Components("bkg_pdf_ext"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kBlue))\ndata_model.plotOn( frame , ROOT.RooFit.Components("bs_pdf_ext"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kBlue))\ndata_model.plotOn(frame, ROOT.RooFit.Name("fit"))\n### Add additional informations to the plot\ntext_size = 0.035\n# Create TLegend\n#legend = ROOT.TLegend(0.7, 0.75, 0.9, 0.9, \'\')\nlegend = ROOT.TLegend(0.7, 0.7,0.85,0.85)\nlegend.AddEntry(0,"LHCb unofficial", "")\nlegend.AddEntry(frame.findObject(\'data1\'), "Data", "p");\nlegend.AddEntry(frame.findObject(\'sig_pdf_ext\'), \'Massmodel\', \'kBluw\')\n\n#legend.AddEntry(frame.findObject(\'bs_pdf_ext\'

In [13]:
#sig_pdf = data_model.getComponents().find('sig_pdf_ext')
#bkg_pdf = data_model.getComponents().find('bkg_pdf_ext')

Number of signal events
---------------------

In [14]:
#sig_yield.getValV(),bkg_yield.getValV(),bs_yield.getValV()

Print fit parameters
-----------------

In [15]:
'''print('a1 =',data_model.getParameters(dataset).find('par_ipatia_a1').getVal())
print('a2 =',data_model.getParameters(dataset).find('par_ipatia_a2').getVal())
print('n1 =',data_model.getParameters(dataset).find('par_ipatia_n1').getVal())
print('n2 =',data_model.getParameters(dataset).find('par_ipatia_n2').getVal())
print('zeta =',data_model.getParameters(dataset).find('par_ipatia_zeta').getVal())
print('sigma =',data_model.getParameters(dataset).find('par_ipatia_sigma').getVal())
print('beta =',data_model.getParameters(dataset).find('par_ipatia_fb').getVal())
print('lambda =',data_model.getParameters(dataset).find('par_ipatia_l').getVal())
print('mass =',data_model.getParameters(dataset).find('par_ipatia_m').getVal())'''

"print('a1 =',data_model.getParameters(dataset).find('par_ipatia_a1').getVal())\nprint('a2 =',data_model.getParameters(dataset).find('par_ipatia_a2').getVal())\nprint('n1 =',data_model.getParameters(dataset).find('par_ipatia_n1').getVal())\nprint('n2 =',data_model.getParameters(dataset).find('par_ipatia_n2').getVal())\nprint('zeta =',data_model.getParameters(dataset).find('par_ipatia_zeta').getVal())\nprint('sigma =',data_model.getParameters(dataset).find('par_ipatia_sigma').getVal())\nprint('beta =',data_model.getParameters(dataset).find('par_ipatia_fb').getVal())\nprint('lambda =',data_model.getParameters(dataset).find('par_ipatia_l').getVal())\nprint('mass =',data_model.getParameters(dataset).find('par_ipatia_m').getVal())"

Calculate sWeights
----------------

In [16]:
'''from ROOT import RooStats
var_sig_yield = data_model.getParameters(dataset).find('sig_yield')
var_bkg_yield = data_model.getParameters(dataset).find('bkg_yield')
data_model_sw = ROOT.RooStats.SPlot("sData","An SPlot", dataset,data_model, RooArgList(var_sig_yield, var_bkg_yield))
#sWeight_sig =ROOT.RooDataSet("dataset_nsig","dataset_nsig", dataset, dataset.get(),"","nsig_sw")
#sWeight_bkg = ROOT.RooDataSet("dataset_nbkg","dataset_nbkg", dataset, dataset.get(),"","nbkg_sw")

#Fill SWeights to numpy array
Weight_sig_array = []
Weight_bkg_array = []
num = sWeight_sig.numEntries()
for i in range(num):
    Weight_sig_array.append(data_sweight.GetSWeight(i,"sig_yield"))
    Weight_bkg_array.append(data_sweight.GetSWeight(i,"bkg_yield"))


from ROOT import TTreeFormula
# writing interim file and tree to have same number of events in ttree and dataset (restricted mass range)
#entries = tree_data.GetEntries()

print('Creatin output file')
interim_file = TFile("/tmp/interim.root","recreate")
interim_tree = tree_data
cut_string = cut
formula = TTreeFormula("formula",cut_string,interim_tree)
interim_tree = tree_data.CopyTree(cut_string)
interim_tree.Write()
data.Close()

# now writing final File
new_file = TFile("/fhgfs/users/chasenberg/data/2015_2016_merged/jpsimumuks/"+file_name_sweight,"recreate")
new_tree = interim_tree.CloneTree()
interim_file.Close()

interim_entries = new_tree.GetEntries()
print("-----------------------")
print("Number of entries in interim tree:")
print(interim_entries)
print("-----------------------")
print("Number of sWeights:")
print(num)


sig_weight = np.zeros(1, dtype=float)
bkg_weight = np.zeros(1, dtype=float)
sigweight_branch = new_tree.Branch('signal_sw',sig_weight,'signal_sw')
bkgweight_branch = new_tree.Branch('background_sw',bkg_weight,'background_sw')

for i in range(0,num):
    sig_weight[0] = Weight_sig_array[i]
    bkg_weight[0] = Weight_bkg_array[i]
    sigweight_branch.Fill()
    bkgweight_branch.Fill()
  
new_tree.Write()
new_file.Close()

# removing interim file
os.remove("/tmp/interim.root")'''

'from ROOT import RooStats\nvar_sig_yield = data_model.getParameters(dataset).find(\'sig_yield\')\nvar_bkg_yield = data_model.getParameters(dataset).find(\'bkg_yield\')\ndata_model_sw = ROOT.RooStats.SPlot("sData","An SPlot", dataset,data_model, RooArgList(var_sig_yield, var_bkg_yield))\n#sWeight_sig =ROOT.RooDataSet("dataset_nsig","dataset_nsig", dataset, dataset.get(),"","nsig_sw")\n#sWeight_bkg = ROOT.RooDataSet("dataset_nbkg","dataset_nbkg", dataset, dataset.get(),"","nbkg_sw")\n\n#Fill SWeights to numpy array\nWeight_sig_array = []\nWeight_bkg_array = []\nnum = sWeight_sig.numEntries()\nfor i in range(num):\n    Weight_sig_array.append(data_sweight.GetSWeight(i,"sig_yield"))\n    Weight_bkg_array.append(data_sweight.GetSWeight(i,"bkg_yield"))\n\n\nfrom ROOT import TTreeFormula\n# writing interim file and tree to have same number of events in ttree and dataset (restricted mass range)\n#entries = tree_data.GetEntries()\n\nprint(\'Creatin output file\')\ninterim_file = TFile("/tmp/in

In [24]:
def copy_ttree_append_sweights(ttree_or_chain, dataset_with_weights):
    sweight_sig = array('d', [0.0])
    sweight_bkg = array('d', [0.0])
        
    sig_sweight_branch = ttree_or_chain.Branch('sweight_sig',sweight_sig,'sweight_sig/D')
    bkg_sweight_branch = ttree_or_chain.Branch('sweight_bkg',sweight_bkg,'sweight_bkg/D')

    for i in range(ttree_or_chain.GetEntries()):
        ttree_or_chain.GetEntry(i)
        control_branch_name = 'B0_FitDaughtersConst_M_flat'
        if (ttree_or_chain.GetBranch('B0_FitDaughtersConst_M_flat').GetLeaf('B0_FitDaughtersConst_M_flat').GetValue() + 0.01 > dataset_with_weights.get(i)['B0_FitDaughtersConst_M_flat'].getValV() and\
            ttree_or_chain.GetBranch('B0_FitDaughtersConst_M_flat').GetLeaf('B0_FitDaughtersConst_M_flat').GetValue() - 0.01 < dataset_with_weights.get(i)['B0_FitDaughtersConst_M_flat'].getValV()):
            # print('Success in Event {}'.format(i))
            sweight_sig[0]=dataset.get(i)['sig_yield_sw'].getValV()
            sweight_bkg[0]=dataset.get(i)['bkg_yield_sw'].getValV()
            sig_sweight_branch.Fill()
            bkg_sweight_branch.Fill()

    return ttree_or_chain

In [25]:
sig_num = []
bkg_num = []
num_all = []

var_sig_yield = data_model.getParameters(dataset).find('sig_yield')
var_bkg_yield = data_model.getParameters(dataset).find('bkg_yield')
sData = RooStats.SPlot('sData', 'sData', dataset, data_model, RooArgList(var_sig_yield,var_bkg_yield))


sig_num.append(var_sig_yield.getValV())
bkg_num.append(var_bkg_yield.getValV())
print('INFO: Finished calculating sWeights', flush=True)

from ROOT import TFile
from array import array
                
f_all_new = TFile("/fhgfs/users/chasenberg/data/2015_2016_merged/jpsimumuks/"+file_name_sweight,"recreate")
tree_w_obscuts = tree_data.CopyTree('')
num_all.append(dataset.numEntries())
copy_ttree_append_sweights(tree_w_obscuts, dataset)
tree_w_obscuts.Write()
f_all_new.Close()

INFO: Finished calculating sWeights


In [ ]:
num_sig = sWeight_sig.numEntries()
num_bkg = sWeight_bkg.numEntries()
#num_tree =tree_data.GetEntries()

print(num_sig,num_bkg)

In [26]:
#cut = '(B0_FitDaughtersConst_M_flat<5450&B0_FitDaughtersConst_M_flat>5220)&B0_FitPVConst_tau_flat>0.0002&B0_FitPVConst_tau_flat<0.015'
#cut = ''
df = rp.read_root("/fhgfs/users/chasenberg/data/2015_2016_merged/jpsimumuks/Bd2JpsiKS_final_sw.root",where=cut, key="Bd2JpsiKS")

In [29]:
sum_signal = df['sweight_sig'].sum()
sum_bkg = df['sweight_bkg'].sum()


print(sum_signal)
print(sum_bkg)
print(sum_signal+sum_bkg)

113666.017195
58528.0450295
172194.062224


In [30]:
df.shape

(173332, 67)